# Creating Custom Tools for LLM Agents with Together AI

This notebook demonstrates how to build custom tools for LLM agents and integrate them with Together AI's Llama 3.1 8B model. We'll create five custom tools:

1. Tool to query the Wikipedia API
2. Tool to query the ArXiv API
3. Tool that converts from Celsius to Fahrenheit
4. Tool that saves the input to a text file
5. Tool that makes a copy of a file

We'll see how the LLM agent uses these tools to answer various queries.

## Setup and Dependencies

First, let's install the necessary dependencies.

In [ ]:
# Install necessary packages
!pip install langchain langchain-together requests langsmith

## Import Libraries

Now we'll import the libraries we need for our custom tools and agent.

In [1]:
import os
import requests
import shutil

from langchain.agents import initialize_agent, Tool
from langchain_together import Together

# Import LangSmith for tracing (optional)
from langsmith import traceable

## API Key Configuration

Set up your Together AI API key. You'll need to get this from [https://together.ai](https://together.ai).

In [ ]:
# Set your Together-AI API key
os.environ["TOGETHER_API_KEY"] = ""

# Optional: Set up LangSmith for tracing (if you have an account)
# os.environ["LANGSMITH_API_KEY"] = "YOUR_LANGSMITH_API_KEY"
# os.environ["LANGCHAIN_TRACE"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = "custom-tools-together"
# os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"  # Set the LangSmith endpoint

## Define Custom Tools

Let's define our five custom tools. Each tool is a function that performs a specific task.

In [3]:
def wikipedia_query(query: str) -> str:
    """
    Tool 1: Query the Wikipedia API.
    Given a topic (query), fetch its summary from Wikipedia.
    """
    # Wikipedia expects the title to be URL-friendly
    title = query.replace(" ", "_")
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("extract", "No summary available.")
    else:
        return f"Error: Unable to retrieve data for '{query}' from Wikipedia."

def arxiv_query(query: str) -> str:
    """
    Tool 2: Query the ArXiv API.
    Returns information about research papers matching the query.
    """
    base_url = "http://export.arxiv.org/api/query"
    params = {
        "search_query": f"all:{query}",
        "start": 0,
        "max_results": 3  # limiting results for brevity
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        # Process the XML to extract useful information
        content = response.text
        # For simplicity, we're returning the raw XML, but you could parse it
        # to extract titles, authors, abstracts, etc.
        return content
    else:
        return f"Error: Unable to query ArXiv for '{query}'."

def celsius_to_fahrenheit(celsius: str) -> str:
    """
    Tool 3: Convert Celsius to Fahrenheit.
    """
    try:
        # Convert input to float
        celsius_value = float(celsius.strip())
        fahrenheit = (celsius_value * 9/5) + 32
        return f"{celsius_value}°C is {fahrenheit}°F."
    except ValueError:
        return f"Error: '{celsius}' is not a valid temperature value."

def save_text_to_file(params: str) -> str:
    """
    Tool 4: Save text to a file.
    Expects input in the format: file_name||content
    """
    try:
        file_name, content = params.split("||", 1)
        with open(file_name.strip(), "w", encoding="utf-8") as f:
            f.write(content.strip())
        return f"Content saved to {file_name.strip()}."
    except Exception as e:
        return f"Error saving file: {str(e)}"

def copy_file(params: str) -> str:
    """
    Tool 5: Copy a file.
    Expects input in the format: source_path||destination_path
    """
    try:
        source_path, destination_path = params.split("||", 1)
        shutil.copyfile(source_path.strip(), destination_path.strip())
        return f"File copied from {source_path.strip()} to {destination_path.strip()}."
    except Exception as e:
        return f"Error copying file: {str(e)}"

## Package Tools for LangChain

Now we'll wrap our custom functions as LangChain Tools that can be used by the agent.

In [4]:
tools = [
    Tool(
        name="wikipedia_query",
        func=wikipedia_query,
        description=(
            "Queries the Wikipedia API to retrieve article summaries. "
            "Input should be the topic title, e.g., 'Artificial Intelligence'."
        )
    ),
    Tool(
        name="arxiv_query",
        func=arxiv_query,
        description=(
            "Queries the ArXiv API to retrieve research papers. "
            "Input should be a search term, e.g., 'machine learning optimization'."
        )
    ),
    Tool(
        name="celsius_to_fahrenheit",
        func=celsius_to_fahrenheit,
        description=(
            "Converts a temperature from Celsius to Fahrenheit. "
            "Input should be a number representing Celsius, e.g., '25'."
        )
    ),
    Tool(
        name="save_text_to_file",
        func=save_text_to_file,
        description=(
            "Saves provided text to a file. "
            "Input should be in the format: 'file_name||content'. "
            "Example: 'greeting.txt||Hello, world!'."
        )
    ),
    Tool(
        name="copy_file",
        func=copy_file,
        description=(
            "Copies a file from a source to a destination. "
            "Input should be in the format: 'source_path||destination_path'. "
            "Example: 'report.doc||backup_report.doc'."
        )
    )
]

## Create the Together AI Agent

Now we'll create an agent using Together AI's Llama 3.1 8B model.

In [5]:
# Initialize the LLM with Together AI
llm = Together(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", # Change the model to the one you want to use
    temperature=0,
    max_tokens=1024
)

# Initialize the agent
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True
)

/var/folders/hy/zvyqxqq95j7_r6sfhrlslk8h0000gn/T/ipykernel_86994/175823869.py:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


## Invoke function (Without Langsmith)

In [6]:
def run_agent(query: str):
    return agent.run(query)

## Optional: Create Traceable Function

If you're using LangSmith for tracing, you can wrap the agent call in a traceable function.

In [ ]:
@traceable
def run_agent(query: str):
    return agent.run(query)

## Test the Agent with Example Queries

Let's test our agent with different queries to see how it uses the custom tools.

### Test 1: Wikipedia Query

Let's see how the agent retrieves information from Wikipedia.

In [7]:
query = "What is the summary of the Wikipedia page for 'Artificial Intelligence'?"
print(f"Query: {query}")
try:
    result = run_agent(query)  # or agent.run(query) if not using tracing
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {str(e)}")

Query: What is the summary of the Wikipedia page for 'Artificial Intelligence'?


> Entering new AgentExecutor chain...


/var/folders/hy/zvyqxqq95j7_r6sfhrlslk8h0000gn/T/ipykernel_86994/775662218.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent.run(query)


 I need to get the summary of the Wikipedia page for 'Artificial Intelligence'.
Action: wikipedia_query
Action Input: Artificial Intelligence
Observation: Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
Thought: I now know the final answer
Final Answer: Artificial intelligence (AI) refers to the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studi

### Test 2: ArXiv Query

Let's see how the agent retrieves research papers from ArXiv.

In [8]:
query = "Find recent research papers on 'large language models' from ArXiv."
print(f"Query: {query}")
try:
    result = run_agent(query)  # or agent.run(query) if not using tracing
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {str(e)}")

Query: Find recent research papers on 'large language models' from ArXiv.


> Entering new AgentExecutor chain...
 I need to find recent research papers on 'large language models' from ArXiv.
Action: arxiv_query
Action Input: 'large language models'
Observation: <?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3Dall%3A%27large%20language%20models%27%26id_list%3D%26start%3D0%26max_results%3D3" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query=all:'large language models'&amp;id_list=&amp;start=0&amp;max_results=3</title>
  <id>http://arxiv.org/api/BrUpOrBD/eWdFFRS8CFA9OLGgO8</id>
  <updated>2025-03-16T00:00:00-04:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1236933</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:items

### Test 3: Temperature Conversion

Let's see how the agent handles temperature conversion.

In [9]:
query = "Convert 25 degrees Celsius to Fahrenheit."
print(f"Query: {query}")
try:
    result = run_agent(query)  # or agent.run(query) if not using tracing
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {str(e)}")

Query: Convert 25 degrees Celsius to Fahrenheit.


> Entering new AgentExecutor chain...
 I need to convert a temperature from Celsius to Fahrenheit.
Action: celsius_to_fahrenheit
Action Input: 25
Observation: 25.0°C is 77.0°F.
Thought: I now know the final answer
Final Answer: 77.0°F. 

> Finished chain.
Result: 77.0°F.


### Test 4: Save Text to File

Let's see how the agent saves text to a file.

In [10]:
query = "Save the text 'Hello, world!' to a file called greeting.txt."
print(f"Query: {query}")
try:
    result = run_agent(query)  # or agent.run(query) if not using tracing
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {str(e)}")

Query: Save the text 'Hello, world!' to a file called greeting.txt.


> Entering new AgentExecutor chain...
 I need to save the text to a file, so I should use the save_text_to_file function.
Action: save_text_to_file
Action Input: 'greeting.txt||Hello, world!'
Observation: Content saved to 'greeting.txt.
Thought: I now know the final answer
Final Answer: Content saved to 'greeting.txt. 

> Finished chain.
Result: Content saved to 'greeting.txt.


### Test 5: Copy File

Let's see how the agent copies a file (assuming greeting.txt was created in the previous step).

In [11]:
query = "Make a copy of greeting.txt to greeting_backup.txt."
print(f"Query: {query}")
try:
    result = run_agent(query)  # or agent.run(query) if not using tracing
    print(f"Result: {result}")
except Exception as e:
    print(f"Error: {str(e)}")

Query: Make a copy of greeting.txt to greeting_backup.txt.


> Entering new AgentExecutor chain...
 I need to copy a file from a source to a destination.
Action: copy_file
Action Input: 'greeting.txt||greeting_backup.txt'
Observation: File copied from 'greeting.txt to greeting_backup.txt'.
Thought: I now know the final answer
Final Answer: File copied from 'greeting.txt to greeting_backup.txt'. 

> Finished chain.
Result: File copied from 'greeting.txt to greeting_backup.txt'.


## Multiple Queries Test

Let's run a batch of queries to test all our tools.

In [ ]:
queries = [
    "What is the summary of the Wikipedia page for 'Deep Learning'?",
    "Find research papers about 'multimodal learning' on ArXiv.",
    "Convert 100 degrees Celsius to Fahrenheit.",
    "Save my shopping list 'Milk, Eggs, Bread' to shopping_list.txt.",
    "Make a copy of shopping_list.txt to shopping_list_backup.txt.",
    "What does the Wikipedia article about 'Machine Learning' say?",
    "Convert -40 degrees Celsius to Fahrenheit."
]

for i, query in enumerate(queries, 1):
    print(f"\n{'-' * 50}\nQuery {i}: {query}")
    try:
        result = run_agent(query)  # or agent.run(query) if not using tracing
        print(f"Result: {result}")
    except Exception as e:
        print(f"Error: {str(e)}")
    print("-" * 50)

## Analysis and Troubleshooting

If the agent isn't selecting the right tools for some queries, you may need to refine the tool descriptions. Here's how you can modify them:

In [ ]:
# Example of refining tool descriptions for better tool selection
refined_tools = [
    Tool(
        name="wikipedia_query",
        func=wikipedia_query,
        description=(
            "Use this tool when you need to get information or summaries from Wikipedia. "
            "Input should be the topic title, e.g., 'Artificial Intelligence'."
        )
    ),
    Tool(
        name="celsius_to_fahrenheit",
        func=celsius_to_fahrenheit,
        description=(
            "Use this tool to convert temperatures from Celsius to Fahrenheit. "
            "Input should ONLY be the numerical value in Celsius (can be negative), e.g., '25' or '-10'."
        )
    ),
    # ... refine other tools as needed
]

## Conclusion

In this notebook, we've created five custom tools and integrated them with Together AI's Llama 3.1 8B model:

1. Wikipedia query tool
2. ArXiv query tool
3. Celsius to Fahrenheit conversion tool
4. Save text to file tool
5. File copy tool

We've seen how the agent selects the appropriate tool based on the query and how to troubleshoot if the tool selection isn't working as expected.

This approach allows you to extend the capabilities of LLM models by giving them access to external functions and APIs, making them more useful for real-world applications.